In [2]:
#저장할 내용을 목록으로 만들어서 엑셀형식과 csv 형식으로 저장하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup     
import pandas as pd   
import openpyxl
import math
import time  
import os
import urllib.request
import urllib

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 네이버 블로그 정보 수집하고 xlsx, csv 형식으로 저장하고 MySQL DB에 저장하기")
print("=" *100)

query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')
cnt=int(input('2.수집할 데이터는 몇 건입니까?: ') )
page_cnt = math.ceil(cnt / 60)

f_dir = input("3.파일을 저장할 폴더명만 쓰세요(기본값:c:/py_temp/):")
if f_dir == '' :
    f_dir="c:\\py_temp\\"
    
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

# Step 3. 크롬 드라이버와 폴더와 파일을 설정하기
s1 = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s1)

#이미지를 저장할 디렉토리 생성
img_dir = f_dir+'Naver_View'+'-'+s+'-'+query_txt+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

# csv , xlsx 파일명 지정
fc_name = f_dir+'Naver_View'+'-'+s+'-'+query_txt+'\\'+'Naver_View'+'-'+s+'-'+query_txt+'.csv'
fx_name = f_dir+'Naver_View'+'-'+s+'-'+query_txt+'\\'+'Naver_View'+'-'+s+'-'+query_txt+'.xlsx'
print('=' *100)

#Step 4. 네이버에 접속 후 검색하고 블로그 선택하기
url = 'https://www.naver.com'
driver.get(url)
driver.maximize_window()
time.sleep(2)

s_time = time.time( )
element = driver.find_element(By.ID,'query')
driver.find_element(By.ID,'query').click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(2)

driver.find_element(By.LINK_TEXT,'VIEW').click( )
time.sleep(1)
driver.find_element(By.LINK_TEXT,'블로그').click( )

# Step 5. 검색 요청 건수만큼 화면 스크롤링 하기
# 자동 스크롤다운 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)

i = 1
while (i <= page_cnt+1):
    scroll_down(driver) 
    i += 1
    print('%s 페이지 정보를 추출하고 있으니 잠시만 기다려 주세요~~^^' %i)

# Step 6. 저장 목록을 만든 후 목록에 있는 내용을 파일에 저장하기

no2 = [ ]           # 게시글 번호 컬럼
title2 = [ ]        # 게시물 제목 컬럼
contents2 = [ ]     # 게시글 내용 컬럼
bdate2 = [ ]        # 작성 일자 컬럼
nick2 = [ ]         # 블로그 닉네임
image2= [ ]         # 이미지 저장 컬럼

no = 1       # 게시글 번호용 숫자
img_no = 1   # 이미지 번호용 숫자

for a in range(1, page_cnt + 1) :
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    blog_list = soup.find('ul','lst_total').find_all('li')
        
    for i in blog_list:
    
        #이미지 수집 후 이미지가 있는 게시글만 추가 정보 수집하기
        try :
            img = i.find('span','thumb_fix').find('img')['src']
        except :
            continue    
        else :
            urllib.request.urlretrieve(img,str(img_no)+'.jpg')
            print('%s번째 게시글의 이미지 저장을 완료했습니다' %img_no)
            print( )        
            time.sleep(1)

            img_no += 1

            # 텍스트 내용 수집
            no2.append(no)                            # 게시물 번호 리스트에 추가
            print('1.번호:',no)

            title_1 = i.find('div','total_area').find_all('a')  
            title = title_1[5].get_text()              # 게시물 제목
            title2.append(title)                       # 게시물 제목 리스트에 추가
            print('2.제목:',title)

            contents = i.find('div','api_txt_lines dsc_txt').get_text( )   # 게시물 요약 내용
            contents2.append(contents)                # 게시물 내용 리스트에 추가
            print('3.요약내용:',contents)

            bdate = i.find('span','sub_time sub_txt').get_text( )  # 작성일자
            bdate2.append(bdate)                     # 작성일자 리스트에 추가
            print('4.작성일자:',bdate)

            nick = i.find('span','elss etc_dsc_inner').get_text( )
            nick2.append(nick)
            print('5.블로그 닉네임:',nick)
            
            image2.append(' ')

            print("\n")

            if no == cnt:
                break

            no += 1

# Step 7.출력 결과를 표(데이터 프레임) 형태로 만들기
naver_blog = pd.DataFrame()
naver_blog['번호'] = no2
naver_blog['제목'] = title2
naver_blog['내용'] = contents2
naver_blog['작성일자'] = bdate2
naver_blog['블로그닉네임'] = nick2

# csv 형태로 저장하기
naver_blog.to_csv(fc_name,encoding="utf-8-sig",index=False)
print("csv 파일 저장 경로: %s" %fc_name) 

# 엑셀 형태로 저장하기
naver_blog.to_excel(fx_name , index=False , encoding="UTF-8" , engine='openpyxl')
print("xls 파일 저장 경로: %s" %fx_name) 

 네이버 블로그 정보 수집하 xlsx, csv 형식으로 저장하고 MySQL DB에 저장하기
1.크롤링할 키워드는 무엇입니까?: 서진수 빅데이터
2.수집할 데이터는 몇 건입니까?: 10
3.파일을 저장할 폴더명만 쓰세요(기본값:c:/py_temp/):
2 페이지 정보를 추출하고 있으니 잠시만 기다려 주세요~~^^
3 페이지 정보를 추출하고 있으니 잠시만 기다려 주세요~~^^
1번째 게시글의 이미지 저장을 완료했습니다

1.번호: 1
2.제목: 전문가에게 길을 묻다. 빅데이터 전문가편 서진수 대표
3.요약내용: 빅데이터 전문가편을 진행하였습니다. 각종 티비 프로그램과 인공지능과 관련된... 등재된 서진수 대표님과 함께 했습니다. 예전에 강연을 잘하셔서 국회의원 상을... 
4.작성일자: 2022.08.06.
5.블로그 닉네임:  뉴턴의과학실&뷰사이언스 


2번째 게시글의 이미지 저장을 완료했습니다

1.번호: 2
2.제목: 마소캠퍼스 빅데이터 분석 오픈 세미나!(서진수 대표님) 151030
3.요약내용: 진행한 빅데이터 분석 오픈 세미나 강연 후기입니다 진행은 서진수 대표님이... 하락중인 데이터를 보고 어떤 인사이트를 얻을 수 있는지, 차례 차례 알려주셨습니다... 
4.작성일자: 2015.10.31.
5.블로그 닉네임:  어쩐지 오늘은 


3번째 게시글의 이미지 저장을 완료했습니다

1.번호: 3
2.제목: 빅데이터 전문가 서진수 대표의 트랜드 분석 강의
3.요약내용: 빅데이터 전문가 서진수 대표님. 빅데이터 강의는 분야를 막론하고 누구든... 미래 빅데이터를 추출하는 것은 기술적으로 그리 어렵지 않다. 오히려 데이터를 어떻게... 
4.작성일자: 2017.03.07.
5.블로그 닉네임:  카루의 프리랜서 라이프 


4번째 게시글의 이미지 저장을 완료했습니다

1.번호: 4
2.제목: 소셜포럼] 서진수 대표, 변화의 흐름을 이끄는 빅데이터... 
3.요약내용: 창의와 융합의 신(新) 물결 ‘4차 산업혁명’ 변화의 흐름을 이끄는 빅데이터 전문가 서진수 ㈜컨

In [4]:
###################################################################################################################
# My SQL DB 관련 작업
###################################################################################################################
# 아래의 작업을 하기 전에 먼저 MySQL DB에서 아래의 작업을 수행해주세요
#[Step 1. 윈도의 명령 프롬프트 창을 실행하여  Mysql 에 접속하기 ]
#C:\WINDOWS\system32>mysql -u root -p
#Enter password: *****
#
#[Step 2.  수집한 데이터를 저장할 데이터 베이스 생성하기 ]
#mysql> CREATE DATABASE crawl_data DEFAULT CHARACTER SET utf8;
#
#[Step 3. 수집한 데이터를 관리할 계정 생성하기]
#mysql> CREATE USER cuser IDENTIFIED BY 'cuser!@#';
#
#[Step 4. 생성한 계정에 권한 설정하기 ]
#mysql> GRANT ALL ON crawl_data.* TO cuser;
####################################################################################################################

#Step 8. MySQL DB에 저장하기
# 접속 정보 등록 및 커서 생성하기
import pymysql
conn = pymysql.connect(host='192.168.150.130', user='cuser', password='cuser!@#', \
                       db='crawl_data', charset='utf8')
cur = conn.cursor()

# 테이블 생성하기
dropsql = "DROP TABLE IF EXISTS naver_blog"
cur.execute(dropsql)

sql = '''CREATE TABLE naver_blog (no char(4), 
         title char(50), content char(200), rdate  char(20) ,nick char(30) )
'''
cur.execute(sql)

# csv 파일 읽어서 테이블에 데이터 입력하기
import csv

sql = "insert into naver_blog (no, title, content, rdate , nick) values (%s, %s, %s,%s , %s)"
file = open(fc_name, 'r', encoding='utf-8')
rdata = csv.reader(file)

for i in rdata:
    cur.execute(sql, (i[0], i[1], i[2] , i[3], i[4]))

conn.commit()
conn.close()
file.close( )

#Step 10. 입력된 데이터 조회하기
import pymysql
conn = pymysql.connect(host='192.168.150.130', user='cuser', password='cuser!@#', \
                       db='crawl_data', charset='utf8')
cur = conn.cursor(pymysql.cursors.DictCursor)

sql = "SELECT * FROM naver_blog"
cur.execute(sql)
result = cur.fetchall()    

import pandas as pd
result2 = pd.DataFrame(result)
result2

,no,title,content,rdate,nick
0,﻿번호,제목,내용,작성일자,블로그닉네임
1,1,전문가에게 길을 묻다. 빅데이터 전문가편 서진수 대표,빅데이터 전문가편을 진행하였습니다. 각종 티비 프로그램과 인공지능과 관련된... 등...,2022.08.06.,뉴턴의과학실&뷰사이언스
2,2,마소캠퍼스 빅데이터 분석 오픈 세미나!(서진수 대표님) 151030,진행한 빅데이터 분석 오픈 세미나 강연 후기입니다 진행은 서진수 대표님이... 하락...,2015.10.31.,어쩐지 오늘은
3,3,빅데이터 전문가 서진수 대표의 트랜드 분석 강의,빅데이터 전문가 서진수 대표님. 빅데이터 강의는 분야를 막론하고 누구든... 미래 ...,2017.03.07.,카루의 프리랜서 라이프
4,4,"소셜포럼] 서진수 대표, 변화의 흐름을 이끄는 빅데이터...",창의와 융합의 신(新) 물결 ‘4차 산업혁명’ 변화의 흐름을 이끄는 빅데이터 전문가...,2017.08.24.,위클리피플 [명사와의 만남] _신지식인 소셜포럼
5,5,"방법, 빅데이터 관리가 가장 기본 <(주)컨시어지소프트 서진수...","된 빅데이터 기술 활용법 알린다” ㈜컨시어지소프트, ㈜데이터앤피플 서진수 대표 4차...",2017.09.25.,월간 인터뷰 (Interview)
6,6,빅데이터 활용으로 4차산업시대를 준비하자_서진수대표 특강...,스몰데이터만으로도 충분히 고객관리가 가능합니다. ^^ 그렇게 지난 날의 관심과 열정...,2017.02.28.,위드앤교육컨설팅_퍼실리테이션 전문교육
7,7,빅데이터 전문가 서진수 강사님의 빅데이터 활용과 4차산업에...,서진수 강사님의 빅데이터 관련 강의를 들으러 강남을 다녀왔드랬죠!! 빅데이터 전문가...,2017.02.27.,한결다온교육연구소
8,8,"빅데이터 분석, 웹크롤링 강남 아이티윌과 서진수 강사님의 특강",오늘은 강남 아이티윌과 서진수 강사님의 빅데이터분석을 위한 R의 활용방안과 텍스트 ...,2017.08.08.,우수훈련기관 아이티윌
9,9,"[모임소개] KBS ""명견만리"" 서진수 직강 '빅데이터 전문가...",(8) 2018.04.22 윈도우10 포스트잇 Sticky Notes 스티커 메모 ...,2018.04.11.,푸우시로의 블로그
